### 一.问题介绍

[本题提供一个数据集, 它包括了5574条英文短信，每条短信内容由几个长短不一的句子组成。每条短信都标注好了是否为垃圾短信，通过该训练集训练出一个分类器，预测短信内容是否为垃圾短信。](https://www.lintcode.com/ai/spam-message-classification/overview)

### 二.数据处理

#### 1.读取数据

对于题目中的csv格式的数据，使用[**`pandas`**](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)中的`pandas.read_csv`函数进行数据读取。cvs中的数据如下：

In [3]:
import pandas as pd
# 读取数据
train_data = pd.read_csv('../Temp/message/train.csv',names = ['Label','Text'])
train_data = train_data.drop(0) # 数据都是字符，无法直接读取，自定name并把第一行删去
test_data = pd.read_csv('../Temp/message/test.csv')

In [4]:
train_data.head() # 训练集的cvs数据呈现

,Label,Text
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,U dun say so early hor... U c already then say...
5,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
test_data.head() # 测试集的cvs数据呈现

,SmsId,Text
0,4456,Aight should I just plan to come up later toni...
1,690,Was the farm open?
2,944,I sent my scores to sophas and i had to do sec...
3,3768,Was gr8 to see that message. So when r u leavi...
4,1189,In that case I guess I'll see you at campus lodge


In [ ]:
对于其Label，我们需要把ham定义成0，把spam定义成1，并把cvs的数据以数组的形式保存，程序如下：

In [11]:
train_data_label = train_data.Label.values # dataframe -> array
for i in range(train_data_label.shape[0]):
    if train_data_label[i] == 'ham':
        train_data_label[i] = 0
    if train_data_label[i] == 'spam':
        train_data_label[i] = 1
        
train_data_content = train_data.Text.values
test_data_content = test_data.Text.values

In [12]:
train_data_label

array([0, 0, 1, ..., 0, 0, 0], dtype=object)

In [13]:
train_data_content

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [14]:
test_data_content

array(['Aight should I just plan to come up later tonight?',
       'Was the farm open?',
       'I sent my scores to sophas and i had to do secondary application for a few schools. I think if you are thinking of applying, do a research on cost also. Contact joke ogunrinde, her school is one me the less expensive ones',
       ...,
       'I will be gentle princess! We will make sweet gentle love...',
       'Beautiful Truth against Gravity.. Read carefully: \\Our heart feels light when someone is in it.. But it feels very heavy when someone leaves it..\\" GOOD NIGHT"',
       'Ups which is 3days also, and the shipping company that takes 2wks. The other way is usps which takes a week but when it gets to lag you may have to bribe nipost to get your stuff.'],
      dtype=object)

#### 2.清洗数据

初步的清洗数据，包括

- 把所有的单词都转换成小写

- 删除除了英文之外的字符

- 把所有的单词恢复成词干

  使用 [`nltk.stem.snowball module`](https://www.nltk.org/api/nltk.stem.html?highlight=nltk%20stem%20snowball#module-nltk.stem.snowball) 用于词干处理

In [15]:
import re
import nltk
from nltk.tokenize import word_tokenize

def clean_text(comment_text):
    comment_list = []
    for text in comment_text:
        # 将单词转换为小写
        text = text.lower()
        # 删除非字母、数字字符
        text = re.sub(r"[^a-z']", " ", text)
        # 恢复常见的简写
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"cannot", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"\'m", " am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " will ", text)
        text = re.sub(r"ain\'t", " are not ", text)
        text = re.sub(r"aren't", " are not ", text)
        text = re.sub(r"couldn\'t", " can not ", text)
        text = re.sub(r"didn't", " do not ", text)
        text = re.sub(r"doesn't", " do not ", text)
        text = re.sub(r"don't", " do not ", text)
        text = re.sub(r"hadn't", " have not ", text)
        text = re.sub(r"hasn't", " have not ", text)
        text = re.sub(r"\'ll", " will ", text)
        #进行词干提取
        new_text = ""
        s = nltk.stem.snowball.EnglishStemmer()  # 英文词干提取器
        for word in word_tokenize(text):
            new_text = new_text + " " + s.stem(word)
        # 放回去
        comment_list.append(new_text)
    return comment_list

train_data_content = clean_text(train_data_content)
test_data_content = clean_text(test_data_content)

#### 3.训练集划分出验证集

因为本题目的测试集中没有label，不便与我们自己验证算法的好坏，因此我们把训练集中70%的数据作为训练集，30%的数据作为验证集

In [36]:
import numpy as np
train_data_content_length = len(train_data_content)
train_data_label_length = len(train_data_label)
train_length = int(train_data_content_length*0.7) 
train_data_content_for_train = train_data_content[0:train_length]  # 取70%的数据作为训练集
train_data_content_for_test = train_data_content[(train_length+1):(train_data_label_length-1)]   # 取30%的数据作为验证集
train_data_label_for_train = train_data_label[0:train_length]  # 取70%的数据作为训练集
train_data_label_for_test = train_data_label[(train_length+1):(train_data_label_length-1)]   # 取30%的数据作为验证集
train_data_label_for_train = train_data_label_for_train.astype(np.float64)
train_data_label_for_test = train_data_label_for_test.astype(np.float64)
print(len(train_data_content_for_train),len(train_data_content_for_test))

3900 1670


#### 4. TF-IDF计算

TF-IDF(Term Frequency-Inverse Document Frequency, 词频-逆文件频率).

> 是一种用于资讯检索与资讯探勘的常用加权技术。TF-IDF是一种统计方法，*用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度*。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

**上述引用总结就是, 一个词语在一篇文章中出现次数越多, 同时在所有文档中出现次数越少, 越能够代表该文章.**

**词频 (term frequency, TF)** 指的是某一个给定的词语在该文件中出现的次数。这个数字通常会被归一化(一般是词频除以文章总词数), 以防止它偏向长的文件。（同一个词语在长文件里可能会比短文件有更高的词频，而不管该词语重要与否。）

> 但是, 需要注意, 一些通用的词语对于主题并没有太大的作用, 反倒是一些出现频率较少的词才能够表达文章的主题, 所以单纯使用是TF不合适的。权重的设计必须满足：一个词预测主题的能力越强，权重越大，反之，权重越小。所有统计的文章中，一些词只是在其中很少几篇文章中出现，那么这样的词对文章的主题的作用很大，这些词的权重应该设计的较大。IDF就是在完成这样的工作.

$$
TF=\frac{在某个文档中词条出现的次数}{该文档的所有词条数目}
$$

**逆向文件频率 (inverse document frequency, IDF)**的主要思想是：如果包含词条t的文档越少, IDF越大，则说明词条具有很好的类别区分能力。某一特定词语的IDF，可以由总文件数目除以包含该词语之文件的数目，再将得到的商取对数得到。
$$
IDF=log(\frac{语料库的文档总数}{包含词条w的文档数+1}),分母之所以要加1，是为了避免分母为0
$$
**某一特定文件内的高词语频率，以及该词语在整个文件集合中的低文件频率，可以产生出高权重的TF-IDF。因此，TF-IDF倾向于过滤掉常见的词语，保留重要的词语。**
$$
TF−IDF=TF∗IDF
$$

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 数据的TF-IDF信息计算
all_comment_list = list(train_data_content_for_train)+list(train_data_content_for_test)+ list(test_data_content)
text_vector = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode',token_pattern=r'\w{1,}',
                              max_features=5000, ngram_range=(1, 1), analyzer='word')
text_vector.fit(all_comment_list)
train_x_for_train = text_vector.transform(train_data_content_for_train)
train_x_for_test = text_vector.transform(train_data_content_for_test)
test_x = text_vector.transform(test_data_content)
train_x_for_train = train_x_for_train.toarray()
train_x_for_test = train_x_for_test.toarray()
test_x = test_x.toarray()
print(train_x_for_train.shape,train_x_for_test.shape,test_x.shape)

(3900, 5000) (1670, 5000) (1115, 5000)


In [23]:
train_x_for_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07808484, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [24]:
train_x_for_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
test_x

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.12213731, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07998202, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### 三.模型选择

采用Logistic Regreesion模型

In [27]:
# 定义sigmoid函数
def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

# 定义似然函数
def log_likelihood(features, target, weights):
    scores = np.dot(features, weights)
    ll = np.sum( target*scores - np.log(1 + np.exp(scores)) )
    return ll

# 定义线性回归模型
def logistic_regression(features, target, num_steps, learning_rate, add_intercept = False):
    if add_intercept:
        intercept = np.ones((features.shape[0], 1))
        features = np.hstack((intercept, features))
        
    weights = np.zeros(features.shape[1])
    
    for step in range(num_steps):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)
        # Update weights with log likelihood gradient
        output_error_signal = target - predictions
        gradient = np.dot(features.T, output_error_signal)
        weights =  weights + learning_rate * gradient

        # Print log-likelihood every so often
        if step % 10000 == 0:
            print(log_likelihood(features, target, weights))
        
    return weights

In [28]:
# 计算权重
weights = logistic_regression(train_x_for_train, train_data_label_for_train,
                     num_steps = 100000, learning_rate = 0.0001, add_intercept=True)

-2502.687115328627
-268.10768148676897
-178.39304483589095
-136.73831728675984
-110.90951212733938
-92.96626612346559
-79.75737737969193
-69.67402522675238
-61.76162440326344
-55.40590986963209


In [29]:
# 显示权重结果
print(weights)

[-4.9149225   3.76204305 -0.04741459 ... -0.01929058  1.01360014
  0.        ]


In [30]:
# 计算准确性
def accuracy(x,label,weights):
    data_with_intercept = np.hstack((np.ones((x.shape[0], 1)),x))
    final_scores = np.dot(data_with_intercept, weights)
    preds = np.round(sigmoid(final_scores))
    print ('Accuracy from scratch: {0}'.format((preds == label).sum().astype(float) / len(preds)))
    
train = accuracy(train_x_for_train,train_data_label_for_train,weights)
train_test = accuracy(train_x_for_train,train_data_label_for_train,weights)

Accuracy from scratch: 0.9994871794871795
Accuracy from scratch: 0.9994871794871795


### 四.导出结果

In [34]:
data_with_intercept = np.hstack((np.ones((test_x.shape[0], 1)),test_x))
final_scores = np.dot(data_with_intercept, weights)
preds = np.round(sigmoid(final_scores))

answer = pd.read_csv(open("../Temp/message/Submission_LR.csv"))
for i in range(answer.shape[0]):
    predit = preds[i]
    if predit ==1:
        answer.loc[i,"Label"] = "spam"
    else:
        answer.loc[i,"Label"] = "ham"
answer.to_csv("../Temp/message/Submission_LR.csv",index=False)  # 不要保存引索列

该导出结果在lintcode上已经可以达到0.99372的准确率